# Pruning in Keras example

## Overview

Welcome to an end-to-end example for magnitude-based *weight pruning*.

### Other pages
For an introduction to what pruning is and to determine if you should use it (including what's supported), see the [overview page](https://www.tensorflow.org/model_optimization/guide/pruning).

To quickly find the APIs you need for your use case (beyond fully pruning a model with 80% sparsity), see the
[comprehensive guide](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md).

### Summary

In this Lab, you will:

1.   Train a `tf.keras` model for MNIST from scratch.
2.   Fine tune the model by applying the pruning API and see the accuracy.
3.   Create 3x smaller TF and TFLite models from pruning.
4.   Create a 10x smaller TFLite model from combining pruning and post-training quantization.
5.   See the persistence of accuracy from TF to TFLite.

## Setup

In [1]:
! pip install -q tensorflow-model-optimization

In [2]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

## Train a model for MNIST without pruning

In [3]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

2022-10-31 09:15:38.541217: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 09:15:38.628735: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5415 - accuracy: 0.8650 - val_loss: 0.1009 - val_accuracy: 0.9738
Epoch 2/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1056 - accuracy: 0.9706 - val_loss: 0.0703 - val_accuracy: 0.9822
Epoch 3/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0737 - accuracy: 0.9788 - val_loss: 0.0624 - val_accuracy: 0.9823
Epoch 4/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0620 - accuracy: 0.9820 - val_loss: 0.0599 - val_accuracy: 0.9828


Evaluate baseline test accuracy and save the model for later usage.

In [4]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9799000024795532
Saved baseline model to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmptll2fl86.h5


## Fine-tune pre-trained model with pruning


### Define the model

You will apply pruning to the whole model and see this in the model summary.

In this example, you start the model with 50% sparsity (50% zeros in weights)
and end with 80% sparsity.

In the [comprehensive guide](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md), you can see how to prune some layers for model accuracy improvements.

In [5]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape  (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


### Train and evaluate the model against baseline

Fine tune with pruning for two epochs.

`tfmot.sparsity.keras.UpdatePruningStep` is required during training, and `tfmot.sparsity.keras.PruningSummaries` provides logs for tracking progress and debugging.

In [6]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2


2022-10-31 09:16:05.212930: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-10-31 09:16:05.212947: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-10-31 09:16:05.212985: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


 26/422 [>.............................] - ETA: 3s - loss: 0.0429 - accuracy: 0.9878

2022-10-31 09:16:06.242151: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-10-31 09:16:06.242172: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-10-31 09:16:06.251894: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-10-31 09:16:06.253725: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-10-31 09:16:06.255770: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmpqf7uiomt/train/plugins/profile/2022_10_31_09_16_06
2022-10-31 09:16:06.256242: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmpqf7uiomt/train/plugins/profile/2022_10_31_09_16_06/Rishabs-MacBook-Pro.local.trace.json.gz
2022-10-31 09:16:06.258318: I tensorflow/core/profiler/rpc/c

422/422 [==============================] - 4s 8ms/step - loss: 0.0637 - accuracy: 0.9826 - val_loss: 0.0983 - val_accuracy: 0.9770
Epoch 2/2
422/422 [==============================] - 3s 7ms/step - loss: 0.0993 - accuracy: 0.9749 - val_loss: 0.0831 - val_accuracy: 0.9773


For this example, there is minimal loss in test accuracy after pruning, compared to the baseline.

In [7]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9799000024795532
Pruned test accuracy: 0.9718999862670898


## Create 3x smaller models from pruning

Both `tfmot.sparsity.keras.strip_pruning` and applying a standard compression algorithm (e.g. via gzip) are necessary to see the compression
benefits of pruning.

*   `strip_pruning` is necessary since it removes every tf.Variable that pruning only needs during training, which would otherwise add to model size during inference
*   Applying a standard compression algorithm is necessary since the serialized weight matrices are the same size as they were before pruning. However, pruning makes most of the weights zeros, which is
added redundancy that algorithms can utilize to further compress the model.

First, create a compressible model for TensorFlow.

In [8]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp6qu97b_4.h5


Then, create a compressible model for TFLite.

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmprnz_zq5k/assets


2022-10-31 09:16:13.151863: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-10-31 09:16:13.311317: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-10-31 09:16:13.311592: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-10-31 09:16:13.312837: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.001ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



Saved pruned TFLite model to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmpykgvuc4u.tflite


2022-10-31 09:16:13.467037: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2022-10-31 09:16:13.467054: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.


Define a helper function to actually compress the models via gzip and measure the zipped size.

In [10]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

Compare and see that the models are 3x smaller from pruning.

In [11]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 78222.00 bytes
Size of gzipped pruned Keras model: 25734.00 bytes
Size of gzipped pruned TFlite model: 24994.00 bytes


## Create a 10x smaller model from combining pruning and quantization

You can apply post-training quantization to the pruned model for additional benefits.

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp36af6tn6/assets


INFO:tensorflow:Assets written to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp36af6tn6/assets


Saved quantized and pruned TFLite model to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmpa5mnu85r.tflite
Size of gzipped baseline Keras model: 78222.00 bytes
Size of gzipped pruned and quantized TFlite model: 8231.00 bytes


2022-10-31 09:16:13.713187: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-10-31 09:16:13.713373: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-10-31 09:16:13.714571: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.001ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-10-31 09:16:13.867600: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2022-10-31 09:16:13.867618: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.
2022-10-31 09:16:13.877801: I tensorflow/lite/tools/optimize/quantize_weights.cc:222] Skipping quantization of tensor sequential/conv2d/Conv2D

## See persistence of accuracy from TF to TFLite

Define a helper function to evaluate the TF Lite model on the test dataset.

In [13]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

You evaluate the pruned and quantized model and see that the accuracy from TensorFlow persists to the TFLite backend.

In [14]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Evaluated on 0 results so far.


INFO: Initialized TensorFlow Lite runtime.


Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Pruned and quantized TFLite test_accuracy: 0.9719
Pruned TF test accuracy: 0.9718999862670898


Problem 1- 


*   Create a model with 3 convolution layer and 3 dense layer for fashion mnist dataset
*   Prune the model to make it sparse then compare the results in terms of size and accuracy

*   Quantize the model after prunning with TFLite then compare the result






In [30]:
fashion = tf.keras.datasets.fashion_mnist.load_data()
(trainX, trainy), (testX, testy) = fashion
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10),
    keras.layers.Dense(10),
    keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Epoch 1/4
1688/1688 [==============================] - 31s 18ms/step - loss: 0.4966 - accuracy: 0.8450 - val_loss: 0.0790 - val_accuracy: 0.9802
Epoch 2/4
1688/1688 [==============================] - 30s 18ms/step - loss: 0.0919 - accuracy: 0.9729 - val_loss: 0.0566 - val_accuracy: 0.9830
Epoch 3/4
1688/1688 [==============================] - 30s 18ms/step - loss: 0.0592 - accuracy: 0.9819 - val_loss: 0.0510 - val_accuracy: 0.9855
Epoch 4/4
1688/1688 [==============================] - 30s 18ms/step - loss: 0.0465 - accuracy: 0.9853 - val_loss: 0.0456 - val_accuracy: 0.9880
Baseline test accuracy: 0.9876999855041504
Saved baseline model to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmpdv6utnlt.h5


In [31]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape_ (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d_4 (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_conv2d_5 (None, 24, 24, 12)        2606      
_________________________________________________________________
prune_low_magnitude_conv2d_6 (None, 22, 22, 12)        2606      
_________________________________________________________________
prune_low_magnitude_max_pool (None, 11, 11, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten_ (None, 1452)              1         
_________________________________________________________________
prune_low_magnitude_dense_6  (None, 10)               

In [32]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp5o00c7an.h5


In [33]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

INFO:tensorflow:Assets written to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp045iyi87/assets


INFO:tensorflow:Assets written to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp045iyi87/assets
2022-10-31 09:38:55.176280: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-10-31 09:38:55.176489: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-10-31 09:38:55.177881: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.001ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.



Saved pruned TFLite model to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmpl9z8fvpx.tflite


2022-10-31 09:38:55.497117: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2022-10-31 09:38:55.497131: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.


In [34]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp7wn7qduv/assets


INFO:tensorflow:Assets written to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp7wn7qduv/assets
2022-10-31 09:38:56.061795: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-10-31 09:38:56.062077: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-10-31 09:38:56.063431: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.



Saved quantized and pruned TFLite model to: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmpm6uzhn_w.tflite
Size of gzipped baseline Keras model: 68042.00 bytes
Size of gzipped pruned and quantized TFlite model: 19144.00 bytes


2022-10-31 09:38:56.395767: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2022-10-31 09:38:56.395790: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.
2022-10-31 09:38:56.410493: I tensorflow/lite/tools/optimize/quantize_weights.cc:222] Skipping quantization of tensor sequential_4/conv2d_4/Conv2D because it has fewer than 1024 elements (108).
2022-10-31 09:38:56.410514: I tensorflow/lite/tools/optimize/quantize_weights.cc:222] Skipping quantization of tensor sequential_4/dense_7/MatMul because it has fewer than 1024 elements (100).
2022-10-31 09:38:56.410517: I tensorflow/lite/tools/optimize/quantize_weights.cc:222] Skipping quantization of tensor sequential_4/dense_8/MatMul because it has fewer than 1024 elements (100).


In [35]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [36]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Pruned and quantized TFLite test_accuracy: 0.9873
Pruned TF test accuracy: 0.9718999862670898


In [37]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

2022-10-31 09:41:34.778084: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-10-31 09:41:34.778155: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-10-31 09:41:34.778325: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


Epoch 1/2
  3/422 [..............................] - ETA: 42s - loss: 0.0256 - accuracy: 0.9913

2022-10-31 09:41:36.031115: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-10-31 09:41:36.031136: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-10-31 09:41:36.139569: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-10-31 09:41:36.142479: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-10-31 09:41:36.145147: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp4dv3_rl3/train/plugins/profile/2022_10_31_09_41_36
2022-10-31 09:41:36.146132: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to /var/folders/g3/jvkl__913hxc658930570qmm0000gn/T/tmp4dv3_rl3/train/plugins/profile/2022_10_31_09_41_36/Rishabs-MacBook-Pro.local.trace.json.gz
2022-10-31 09:41:36.149210: I tensorflow/core/profiler/rpc/c

422/422 [==============================] - 27s 60ms/step - loss: 0.0936 - accuracy: 0.9697 - val_loss: 0.1189 - val_accuracy: 0.9648
Epoch 2/2
422/422 [==============================] - 27s 63ms/step - loss: 0.1121 - accuracy: 0.9670 - val_loss: 0.0727 - val_accuracy: 0.9787


Problem 2 - Use pretrained VGG16 model prune and quantize the model for Fashion Mnist Dataset and compare the results with original vs pruned vs quantize

In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, time
import matplotlib.pyplot as plt
#from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
#from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16;
from keras.applications.vgg16 import preprocess_input
import os

In [48]:
trainX.shape, testX.shape

((60000, 28, 28), (10000, 28, 28))

In [49]:
trainX=np.dstack([trainX] * 3)
testX=np.dstack([testX]*3)
trainX.shape,testX.shape

((60000, 28, 84), (10000, 28, 84))

In [51]:
# Reshape images as per the tensor format required by tensorflow
train_X = trainX.reshape(-1, 28,28,3)
test_X= testX.reshape (-1,28,28,3)
train_X.shape,test_X.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

In [52]:
from keras.preprocessing.image import img_to_array, array_to_img
train_X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in train_X])
test_X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in test_X])
#train_x = preprocess_input(x)
train_X.shape, test_X.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

In [53]:
train_X = train_X / 255.
test_X = test_X / 255.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

In [56]:
train_Y_one_hot = to_categorical(trainy)
test_Y_one_hot = to_categorical(testy)

In [57]:
train_X,valid_X,train_label,valid_label = train_test_split(train_X,
                                                           train_Y_one_hot,
                                                           test_size=0.2,
                                                           random_state=13
                                                           )

In [58]:
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

((48000, 48, 48, 3), (12000, 48, 48, 3), (48000, 10), (12000, 10))

In [59]:
IMG_WIDTH = 48
IMG_HEIGHT = 48
IMG_DEPTH = 3
BATCH_SIZE = 16

In [60]:
train_X = preprocess_input(train_X)
valid_X = preprocess_input(valid_X)
test_X  = preprocess_input (test_X)

In [64]:
conv_base = VGG16(weights='imagenet',
                  include_top=False, 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)
                 )
conv_base.summary()

58892288/58889256 [==============================] - 49s 1us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_____________________________________________________________

In [ ]:
train_features = conv_base.predict(np.array(train_X), batch_size=BATCH_SIZE, verbose=1)
test_features = conv_base.predict(np.array(test_X), batch_size=BATCH_SIZE, verbose=1)

 235/3000 [=>............................] - ETA: 8:20